In [1]:
'''
20190112 機能追加「各地域版を個別に選択する」
'''

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import pandas as pd

#search_keyword="GMO"

articles=[]
#f=open("C_YomiuriChecker1.txt","w",encoding="utf-8")

#一覧表示&テキストスクレイピングのプログラム
def newsdownload(browser,nl):
    browser.find_element_by_xpath('//*[@onclick="toggleCheck(document.forms[\'search\'], \'yomiuriNewsArticleDto.chkboxCollectiveIndication\', true);return false;"]').click()
    browser.find_element_by_xpath('//*[@onclick="validateAllDisplayCheckBox(\'yomiuriNewsArticleDto.chkboxCollectiveIndication\', \'search\', new Array(\'表示チェックが選択されていません。\', \'一括表示できる件数は20件までです。続行しますか？\'), \'yomiuriNewsArticle.action?allDisplayFlg=true\');return false;"]').click()
    datelist=browser.find_elements_by_xpath('//*[@style="width:80px;vertical-align:middle;"]')
    alllist=browser.find_elements_by_xpath('//table[@class="contentsTable"]/tbody/tr')
    daynightlist,arealist,pagelist,sizelist=[],[],[],[]
    for tmps in alllist:
        tmps=str(tmps.text)
        tlist=tmps.split(" ")
        daynightlist.append(tlist[3])
        arealist.append(tlist[4])
        pagelist.append(tlist[5])
        sizelist.append(tlist[6])

    ttllist=browser.find_elements_by_class_name("wp50")
    newslist=browser.find_elements_by_class_name("mb10")
    for date,ttl,news,daynight,area,page,size in zip(datelist,ttllist,newslist,daynightlist,arealist,pagelist,sizelist):
        #if daynight.startswith("東京朝刊") or daynight.startswith("東京夕刊"):
            #print(daynight.text)
            #print(date.text)
            #print(ttl.text)
            #print(news.text)
            #f.write("<H1>"+date.text+"</H1>"+"\n")
            #f.write("<H2>"+ttl.text+"</H2>"+"\n")
            #f.write("\n"+news.text+"\n\n\n")
            #print("■■■■■■■■■■■■■■■■■■")
        nl.append([date.text,ttl.text,daynight,area,page,size,news.text])
        #else:
        #    pass
    time.sleep(3)
    browser.find_element_by_xpath('//*[@onclick="execute(document.forms[\'article\'], \'yomiuriNewsPageSearchList.action\');return false;"]').click()
    time.sleep(1)
    browser.find_element_by_class_name("nextPage").click()

#早稲田検索ページを開ける
def getpage(browser,search_keyword,newsdata):
    #browser.get("http://waseda.summon.serialssolutions.com/jp/search?q=%E8%AA%AD%E5%A3%B2%E6%96%B0%E8%81%9E&l=jp#!/search?ho=t&l=jp&q=%E8%AA%AD%E5%A3%B2%E6%96%B0%E8%81%9E")
    #time.sleep(3)
    #targetlink=browser.find_element_by_xpath('//*[@ng-bind="::item.bet.title"]')
    #targetlink.click()
    #print("1.検索完了")
    #新しいページでログインする
    #allHandles = browser.window_handles
    #for handle in allHandles:
    #    if browser.title.find("EZproxy") == -1:
    #        browser.switch_to_window(handle)
    #elem_user = browser.find_element_by_name("user")
    #elem_user.send_keys("tsh")
    #elem_pwd = browser.find_element_by_name("pass")
    #elem_pwd.send_keys("Aragaki01!")
    #browser.find_element_by_xpath('//*[@type="submit"]').click()
    browser.get("https://database.yomiuri.co.jp/rekishikan/")
    time.sleep(2)
    #print("2.ログイン完了")
    #読売歴史館に入った後、検索モードを平成モードに切り替える
    browser.get("https://database-yomiuri-co-jp.ez.wul.waseda.ac.jp/rekishikan/yomiuriNewsSearch.action")
    time.sleep(1)
    #検索キーワードを入力する
    elem_inkw = browser.find_element_by_name("yomiuriNewsSearchDto.txtWordSearch")
    elem_inkw.send_keys(search_keyword)
    
    browser.find_element_by_xpath('//*[@value="100"]').click() #100件表示

    browser.find_element_by_class_name("choiceArea1Open").click()  #個別に選択する
    browser.find_element_by_xpath(u'//*[@title="全国版"]').click() #全国版
    #browser.find_element_by_xpath(u'//*[@title="地域版"]').click() #地域版
    
    #各地域版を個別に選択する
    #browser.find_element_by_xpath(u'//*[@title="各地域版を個別に選択する"]').click()
    ##関東
    #browser.find_element_by_xpath(u'//*[@title="茨城"]').click()
    #browser.find_element_by_xpath(u'//*[@title="栃木"]').click()
    #browser.find_element_by_xpath(u'//*[@title="群馬"]').click()
    #browser.find_element_by_xpath(u'//*[@title="埼玉"]').click()
    #browser.find_element_by_xpath(u'//*[@title="千葉"]').click()
    #browser.find_element_by_xpath(u'//*[@title="東京"]').click()
    #browser.find_element_by_xpath(u'//*[@title="神奈川"]').click()
    
    #期間
    browser.find_element_by_id("txtSYear").send_keys("2008")
    browser.find_element_by_id("txtSMonth").send_keys("1")
    browser.find_element_by_id("txtSDay").send_keys("1")
    browser.find_element_by_id("txtEYear").send_keys("2018")
    browser.find_element_by_id("txtEMonth").send_keys("12")
    browser.find_element_by_id("txtEDay").send_keys("31")
    time.sleep(5)
    browser.find_element_by_id("yomiuriNewsSubmitButton").click()
    #print("3.検索完了")
    #print("■■■■■■■■■■■■■■■■■■")

    #ページの切り替え
    w_start=1
    w_news=browser.find_element_by_xpath('//*[@class="heiseiOperationLeft"]//*[@class="flL"]')
    w_news=w_news.text.replace(" 件中","")
    w_page=int(w_news)//100+1
    while w_start<=w_page:
        newsdownload(browser,newsdata)
        w_start+=1
#print("4.done")

#keywordlist=["(ワーキングママ OR ワーキング母 OR ワーママ OR ワーキングマザー OR 働くママ OR 働く母) AND (子ども OR 子育て OR 保育 OR 育児 OR ＰＴＡ OR 児童 OR 保育園 OR 出産 OR 育休 OR 子供 OR 主婦 OR 家事 OR 職場 OR 改革 OR キャリア OR 妊娠 OR 共働き OR 結婚 OR 産む OR 赤ちゃん)"]
#keywordlist=["［戦後７０年",
#             "［社会保障　戦後７０年］",
#             "［証言　大連の抑留］",
#             "［７０年後の返信］",
#             "［ニッポンの貢献　戦後７０年］",
#             "［語る］戦後７０年",
#             "［日米関係　戦後７０年］"]
keywordlist=["人工知能"]

browser = webdriver.Chrome()
for keyword in keywordlist:
    getpage(browser,keyword,articles)
browser.quit()
#データフレーム構築
df = pd.DataFrame(articles,columns=["時間","タイトル","地域","面","ページ","文字数","記事内容"])
#df.to_csv("C_YomiuriChecker1.csv",encoding="utf-8",sep="\t", index=False)
df.to_excel('YomiuriCheck_AI.xlsx', sheet_name='sheet1')
df

,時間,タイトル,地域,面,ページ,文字数,記事内容
0,2018.12.31,法律翻訳にＡＩ活用へ 「言い回し」蓄積→処理速度向上 有識者会議で検討,東京朝刊,二面,02頁,738字,法務省は、日本の法律を効率よく英語に翻訳するため、人工知能（ＡＩ）を活用する方向で検討に入っ...
1,2018.12.30,平成時代 深掘り世論調査特集（上）ネット社会 光と影 情報 豊かさの象徴に,東京朝刊,朝特Ｄ,08頁,4326字,◆新「三種の神器」 暮らし一変\n■１９９５年の期待\n私たちはインターネットの普及により、...
2,2018.12.30,［編集手帳］１２月３０日,東京朝刊,一面,01頁,469字,新年の季語である嫁が君は、正月にネズミを指すときの忌み言葉で、地域によっては福の神とか、福太...
3,2018.12.30,ＡＩで安心の街づくり ＮＴＴ、米で事業拡大へ,東京朝刊,政経,04頁,420字,ＮＴＴの澤田純社長は読売新聞のインタビューに応じ、米国で人工知能（ＡＩ）を使って都市の犯罪や...
4,2018.12.29,中国 ベンチャー育成に注力 アニス・ウッザマン氏４３,東京朝刊,解説,09頁,2772字,◇フェノックス・ベンチャー・キャピタルＣＥＯ\n◆地方政府 成長へ投資活発\n中国では今、国...
5,2018.12.29,株 高速取引で乱高下 ２０１８年日米相場 ＡＩ ニュース連動で即売買,東京朝刊,Ａ経,08頁,1181字,２０１８年は日米ともに株価が乱高下した。トランプ米大統領といった要人の発言や、経済データに人...
6,2018.12.28,［論点］公共政策 ＡＩが開く未来 広井良典氏（寄稿）,東京朝刊,解説,11頁,1267字,近年、あらゆる場面で「ＡＩ（人工知能）」という言葉を見聞きするようになった。私は１９８０年代...
7,2018.12.28,［いのち守る］児童相談所の今（中）「特別な日」毎年メール（連載）,東京朝刊,２社,28頁,1523字,◆過去データ蓄積 判断の助け\n「明日は児童相談所（児相）にとって特別な日です。失われた子ど...
8,2018.12.28,新たな１００年 米中へ軸足 合計売上高 日本の水準目指す パナソニック,大阪朝刊,Ａ経,07頁,787字,パナソニックの津賀一宏社長は読売新聞のインタビューに応じ、２０３０年に米国と中国での合計売上...
9,2018.12.27,「官製春闘」転換点に 首相賃上げ要請 財界意向に配慮,東京朝刊,Ａ経,09頁,954字,政府主導の「官製春闘」が転換点を迎えつつある。安倍首相は２０１９年春闘で、経団連に具体的な賃...
